In [ ]:
#Query 1

In [30]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "1g",
        "spark.executor.cores": "1",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
27,application_1738075734771_0028,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1738075734771_0012,pyspark,idle,Link,Link,None,
14,application_1738075734771_0015,pyspark,idle,Link,Link,None,
15,application_1738075734771_0016,pyspark,idle,Link,Link,None,
18,application_1738075734771_0019,pyspark,idle,Link,Link,None,
21,application_1738075734771_0022,pyspark,idle,Link,Link,None,
22,application_1738075734771_0023,pyspark,idle,Link,Link,None,
26,application_1738075734771_0027,pyspark,idle,Link,Link,None,
27,application_1738075734771_0028,pyspark,idle,Link,Link,None,✔


In [31]:

# Spark RDD code
from pyspark.sql import SparkSession
#To log our application's execution time:
import time
import csv
from io import StringIO

sc = SparkSession \
    .builder \
    .appName("RDD query 1 execution") \
    .getOrCreate() \
    .sparkContext

start_time = time.time()

#crime data
#When using the given format the csv file was wrongly separated and it gave wrong records. That's why we created a function that is able to open the csv with respect to 
#commas inside columns.

#cd_2010_2019 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv") \
#                   .map(lambda x: (x.split(",")))
#cd_2020_ = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv") \
#                    .map(lambda x: (x.split(",")))

def parse_csv(line):
    return next(csv.reader(StringIO(line)))

cd_2010_2019 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv") \
    .map(parse_csv)

cd_2020_ = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv") \
    .map(parse_csv)

header_2010_2019 = cd_2010_2019.first()
header_2020 = cd_2020_.first()

data_2010_2019 = cd_2010_2019.filter(lambda x: x != header_2010_2019)
data_2020 = cd_2020_.filter(lambda x: x != header_2020)

#Filter to get only the incidents with aggravated assault
cd_aggr_2010_2019 = data_2010_2019.filter(lambda x: "AGGRAVATED ASSAULT" in x[9])
cd_aggr_2020_ = data_2020.filter(lambda x: "AGGRAVATED ASSAULT" in x[9])

#For each category, adjust the age limits, then combine the data from both files and finally sort them on descending order by Victim Age
cd_kids_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) < 18 and int(x[11]) >= 0)
cd_kids_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) < 18 and int(x[11]) >= 0)
cd_kids = cd_kids_2010_2019.union(cd_kids_2020_)
sorted_cd_kids = cd_kids.sortBy(lambda x: x[11], ascending=False)

cd_young_adlt_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) >= 18 and int(x[11]) < 25)
cd_young_adlt_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) >= 18 and int(x[11]) < 25)
cd_young_adlt = cd_young_adlt_2010_2019.union(cd_young_adlt_2020_)
sorted_cd_young_adlt = cd_young_adlt.sortBy(lambda x: x[11], ascending=False)

cd_adlt_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) >= 25 and int(x[11]) <= 64)
cd_adlt_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) >= 25 and int(x[11]) <= 64)
cd_adlt = cd_adlt_2010_2019.union(cd_adlt_2020_)
sorted_cd_adlt = cd_adlt.sortBy(lambda x: x[11], ascending=False)

cd_eldery_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) > 64)
cd_eldery_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) > 64)
cd_eldery = cd_eldery_2010_2019.union(cd_eldery_2020_)
sorted_cd_eldery = cd_eldery.sortBy(lambda x: x[11], ascending=False)

end_time = time.time()
ex_time=end_time - start_time

# Take a sample of each RDD
#print("Sample of sorted_cd_kids:")
#print(sorted_cd_kids.take(10))

#print("Sample of sorted_cd_young_adlt:")
#print(sorted_cd_young_adlt.take(10))

#print("Sample of sorted_cd_adlt:")
#print(sorted_cd_adlt.take(10))

#print("Sample of sorted_cd_eldery:")

#print(sorted_cd_eldery.take(10))

print(f"Number of rows in the Kids: {sorted_cd_kids.count()}")
print(f"Number of rows in the Young Adults: {sorted_cd_young_adlt.count()}")
print(f"Number of rows in the Adults: {sorted_cd_adlt.count()}")
print(f"Number of rows in the Eldery: {sorted_cd_eldery.count()}")

print(f"Time taken: {ex_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of rows in the Kids: 15923
Number of rows in the Young Adults: 33605
Number of rows in the Adults: 121093
Number of rows in the Eldery: 5985
Time taken: 50.12 seconds

In [32]:
#Query 1

# Spark DataFrame code
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col
import time
spark = SparkSession \
    .builder \
    .appName("DF query 1 execution") \
    .getOrCreate()

crimedata = StructType([
    StructField("DR_NO", StringType(), True),
    StructField("Date Rptd", StringType(), True),
    StructField("DATE OCC", StringType(), True),
    StructField("TIME OCC", StringType(), True),
    StructField("AREA", IntegerType(), True),
    StructField("AREA NAME", StringType(), True),
    StructField("Rpt Dist No", IntegerType(), True),
    StructField("Part 1-2", IntegerType(), True),
    StructField("Crm Cd", IntegerType(), True),
    StructField("Crm Cd Desc", StringType(), True),
    StructField("Mocodes", StringType(), True),
    StructField("Vict Age", IntegerType(), True),
    StructField("Vict Sex", StringType(), True),
    StructField("Vict Descent", StringType(), True),
    StructField("Premis Cd", IntegerType(), True),
    StructField("Premis Desc", StringType(), True),
    StructField("Weapon Used Cd", IntegerType(), True),
    StructField("Weapon Desc", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Status Desc", StringType(), True),
    StructField("Crm Cd 1", IntegerType(), True),
    StructField("Crm Cd 2", IntegerType(), True),
    StructField("Crm Cd 3", IntegerType(), True),
    StructField("Crm Cd 4", IntegerType(), True),
    StructField("LOCATION", StringType(), True),
    StructField("Cross Street", StringType(), True),
    StructField("LAT", FloatType(), True),
    StructField("LON", FloatType(), True)
])

schema1 = "Crm_Cd_Desc STRING, Vict_Age INT"

# Starting Time
start_time = time.time()

crimedata_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=False, schema=crimedata)
crimedata_2020_ = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=False, schema=crimedata)

#Filter to get only the incidents with aggravated assault
crimedata_2010_2019_aggr = crimedata_2010_2019.filter(col("Crm Cd Desc").contains("AGGRAVATED ASSAULT"))
crimedata_2020_aggr = crimedata_2020_.filter(col("Crm Cd Desc").contains("AGGRAVATED ASSAULT"))

#For each category, adjust the age limits, then combine the data from both files and finally sort them on descending order by Victim Age
kids2010_2019 = crimedata_2010_2019_aggr.filter((col("Vict Age") >= 0) & (col("Vict Age") < 18))
kids2020_ = crimedata_2020_aggr.filter((col("Vict Age") >= 0) & (col("Vict Age") < 18))
kids = kids2010_2019.union(kids2020_)
sorted_kids = kids.sort(col("Vict Age"), ascending=False)
sorted_kids = kids.orderBy(col("Vict Age").desc())

young_adult2010_2019 = crimedata_2010_2019_aggr.filter((col("Vict Age") >= 18) & (col("Vict Age") <= 24))
young_adult2020_ = crimedata_2020_aggr.filter((col("Vict Age") >= 18) & (col("Vict Age") <= 24))
young_adults = young_adult2010_2019.union(young_adult2020_)
sorted_young_adults = young_adults.orderBy(col("Vict Age").desc())

adult2010_2019 = crimedata_2010_2019_aggr.filter((col("Vict Age") >= 25) & (col("Vict Age") <= 64))
adult2020_ = crimedata_2020_aggr.filter((col("Vict Age") >= 25) & (col("Vict Age") <= 64))
adults = adult2010_2019.union(adult2020_)
sorted_adults = adults.orderBy(col("Vict Age").desc())

elderly2010_2019 = crimedata_2010_2019_aggr.filter(col("Vict Age") > 64)
elderly2020_ = crimedata_2020_aggr.filter(col("Vict Age") > 64)
elderly = elderly2010_2019.union(elderly2020_)
sorted_elderly = elderly.orderBy(col("Vict Age").desc())

#Show how much time did it take
end_time = time.time()
ex_time = end_time - start_time

#Show results
#sorted_kids.show(10)
#sorted_young_adults.show(10)
#sorted_adults.show(10)
#sorted_elderly.show(10)

print(f"Number of rows in the Kids: {sorted_kids.count()}")
print(f"Number of rows in the Young Adults: {sorted_young_adults.count()}")
print(f"Number of rows in the Adults: {sorted_adults.count()}")
print(f"Number of rows in the Eldery: {sorted_elderly.count()}")
print(f"Time taken: {ex_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of rows in the Kids: 15923
Number of rows in the Young Adults: 33605
Number of rows in the Adults: 121093
Number of rows in the Eldery: 5985
Time taken: 2.34 seconds

In [63]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
92,application_1738075734771_0093,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
83,application_1738075734771_0084,pyspark,idle,Link,Link,None,
86,application_1738075734771_0087,pyspark,idle,Link,Link,None,
88,application_1738075734771_0089,pyspark,idle,Link,Link,None,
89,application_1738075734771_0090,pyspark,idle,Link,Link,None,
90,application_1738075734771_0091,pyspark,idle,Link,Link,None,
91,application_1738075734771_0092,pyspark,idle,Link,Link,None,
92,application_1738075734771_0093,pyspark,idle,Link,Link,None,✔


In [34]:
# Spark RDD code
from pyspark.sql import SparkSession
# To log our application's execution time:
import time

sc = SparkSession \
    .builder \
    .appName("RDD query 1 execution") \
    .getOrCreate() \
    .sparkContext

start_time = time.time()
#crime data
cd_2010_2019 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv") \
                    .map(lambda x: (x.split(",")))
cd_2020_ = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv") \
                    .map(lambda x: (x.split(",")))

header_2010_2019 = cd_2010_2019.first()
header_2020 = cd_2020_.first()

data_2010_2019 = cd_2010_2019.filter(lambda x: x != header_2010_2019)
data_2020 = cd_2020_.filter(lambda x: x != header_2020)

cd_aggr_2010_2019 = data_2010_2019.filter(lambda x: "AGGRAVATED ASSAULT" in x[9])
cd_aggr_2020_ = data_2020.filter(lambda x: "AGGRAVATED ASSAULT" in x[9])

cd_kids_2010_2019 = cd_aggr_2010_2019.filter(lambda x: x[11] < "18" and x[11] >= "0")
cd_kids_2020_ = cd_aggr_2020_.filter(lambda x: x[11] < "18" and x[11] > "0")
cd_kids = cd_kids_2010_2019.union(cd_kids_2020_)
sorted_cd_kids = cd_kids.sortBy(lambda x: x[11], ascending=False)

cd_young_adlt_2010_2019 = cd_aggr_2010_2019.filter(lambda x: int(x[11]) >= 18 and int(x[11]) < 25)
cd_young_adlt_2020_ = cd_aggr_2020_.filter(lambda x: int(x[11]) >= 18 and int(x[11]) < 25)
cd_young_adlt = cd_young_adlt_2010_2019.union(cd_young_adlt_2020_)
sorted_cd_young_adlt = cd_young_adlt.sortBy(lambda x: x[11], ascending=False)

cd_adlt_2010_2019 = cd_aggr_2010_2019.filter(lambda x: x[11] >= "25" and x[11] <= "64")
cd_adlt_2020_ = cd_aggr_2020_.filter(lambda x: x[11] >= "25" and x[11] <= "64")
cd_adlt = cd_adlt_2010_2019.union(cd_adlt_2020_)
sorted_cd_adlt = cd_adlt.sortBy(lambda x: x[11], ascending=False)

cd_eldery_2010_2019 = cd_aggr_2010_2019.filter(lambda x: x[11] > "64")
cd_eldery_2020_ = cd_aggr_2020_.filter(lambda x: x[11] > "64")
cd_eldery = cd_eldery_2010_2019.union(cd_eldery_2020_)
sorted_cd_eldery = cd_eldery.sortBy(lambda x: x[11], ascending=False)

end_time = time.time()
time_ex=end_time - start_time

print("Sample of sorted_cd_young_adlt:")
print(sorted_cd_young_adlt.take(10))

print("Sample of sorted_cd_adlt:")
print(sorted_cd_adlt.take(10))

print("Sample of sorted_cd_eldery:")
print(sorted_cd_eldery.take(10))

# Take a sample of each RDD
print("Sample of sorted_cd_kids:")
print(sorted_cd_kids.take(10))
print(time_ex)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sample of sorted_cd_young_adlt:
[['180200700', '03/31/2018 12:00:00 AM', '03/31/2018 12:00:00 AM', '0330', '02', 'Rampart', '0204', '1', '236', 'INTIMATE PARTNER - AGGRAVATED ASSAULT', '0417 0416 0443 2000 1813 1310 0429 1206', '24', 'F', 'H', '502', '"MULTI-UNIT DWELLING (APARTMENT', ' DUPLEX', ' ETC)"', '400', '"STRONG-ARM (HANDS', ' FIST', ' FEET OR BODILY FORCE)"', 'AA', 'Adult Arrest', '236', '', '', '', '700    IMOGEN                       AV', '', '34.0822', '-118.2832'], ['181508373', '03/26/2018 12:00:00 AM', '03/25/2018 12:00:00 AM', '0340', '15', 'N Hollywood', '1532', '1', '236', 'INTIMATE PARTNER - AGGRAVATED ASSAULT', '2000 1814 0444', '24', 'F', 'H', '502', '"MULTI-UNIT DWELLING (APARTMENT', ' DUPLEX', ' ETC)"', '400', '"STRONG-ARM (HANDS', ' FIST', ' FEET OR BODILY FORCE)"', 'AA', 'Adult Arrest', '236', '', '', '', '6100    WHITSETT                     AV', '', '34.1813', '-118.4052'], ['180325257', '11/11/2018 12:00:00 AM', '11/11/2018 12:00:00 AM', '1400', '03', 'Sout

In [35]:
# Spark DataFrame code
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col

spark = SparkSession \
    .builder \
    .appName("DF query 1 execution") \
    .getOrCreate()

employees_schema = StructType([
    StructField("id", IntegerType()),
    StructField("name", StringType()),
    StructField("salary", FloatType()),
    StructField("dep_id", IntegerType()),
])

employees_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/spark-example-data/employees.csv", header=False, schema=employees_schema)
#print the schema of the DataFrame:
employees_df.printSchema()

## Alternative way to read csv:
# employees_df = spark.read.format('csv') \
#     .options(header='false') \
#     .schema(employees_schema) \
#     .load("s3://initial-notebook-data-bucket-dblab-905418150721/spark-example-data/employees.csv")

sorted_employees_df = employees_df.sort(col("salary"))
sorted_employees_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- dep_id: integer (nullable = true)

+---+---------+------+------+
| id|     name|salary|dep_id|
+---+---------+------+------+
|  6|  Jerry L| 550.0|     3|
|  2|   John K|1000.0|     2|
|  7| Marios K|1000.0|     1|
|  5|  Helen K|1050.0|     2|
| 10|Yiannis T|1500.0|     1|
+---+---------+------+------+
only showing top 5 rows

In [ ]:
#QUERY 2 WITH DATAFRAMES

In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, dense_rank
from pyspark.sql.window import Window
import time

sc = SparkSession \
    .builder \
    .appName("Query 2") \
    .getOrCreate()


crime_df_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
crime_df_2020_present = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)
# Ενοποίηση των δύο DataFrames

crime_df = crime_df_2010_2019.union(crime_df_2020_present)
crime_df = crime_df.select(
    col("DATE OCC").alias("date_occ"),
    col("AREA NAME").alias("precinct"),
    col("Status Desc").alias("status")
)
crime_df = crime_df.withColumn("year", col("date_occ").substr(7, 4))
crime_closed = crime_df.filter(col("status").isin("Adult Arrest", "Adult Other", "Juv Arrest", "Juv Other"))

crime_stats = crime_closed.groupBy("year", "precinct") \
    .agg(
        count("*").alias("closed_cases")
    )
total_cases = crime_df.groupBy("year", "precinct") \
    .agg(
        count("*").alias("total_cases")
    )
crime_percentage = crime_stats.join(total_cases, ["year", "precinct"]) \
    .withColumn("closed_case_rate", (col("closed_cases") / col("total_cases")) * 100)
window_spec = Window.partitionBy("year").orderBy(desc("closed_case_rate"))
crime_top3 = crime_percentage.withColumn("rank", dense_rank().over(window_spec)) \
    .filter(col("rank") <= 3)

crime_top3_sorted = crime_top3.orderBy("year", "rank")

start_time = time.time()
crime_top3_sorted.select("year", "precinct", "closed_case_rate", "rank").show()
end_time = time.time()
dataframe_api_time = end_time - start_time
print(f"Time taken by DataFrame API: {dataframe_api_time} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|year|   precinct|  closed_case_rate|rank|
+----+-----------+------------------+----+
|2010|    Rampart| 32.84713448949121|   1|
|2010|    Olympic|31.515289821999087|   2|
|2010|     Harbor| 29.36028339237341|   3|
|2011|    Olympic|35.040060090135206|   1|
|2011|    Rampart|  32.4964471814306|   2|
|2011|     Harbor| 28.51336246316431|   3|
|2012|    Olympic| 34.29708533302119|   1|
|2012|    Rampart| 32.46000463714352|   2|
|2012|     Harbor|29.509585848956675|   3|
|2013|    Olympic| 33.58217940999398|   1|
|2013|    Rampart|  32.1060382916053|   2|
|2013|     Harbor|29.723638951488557|   3|
|2014|   Van Nuys|  32.0215235281705|   1|
|2014|West Valley| 31.49754809505847|   2|
|2014|    Mission|31.224939855653567|   3|
|2015|   Van Nuys|32.265140677157845|   1|
|2015|    Mission|30.463762673676303|   2|
|2015|   Foothill|30.353001803658852|   3|
|2016|   Van Nuys|32.194518462124094|   1|
|2016|West Valley| 31.40146437042384|   2|
+----+-----

In [ ]:
#QUERY 2 WITH SQL API

In [37]:
# Register the DataFrames as temporary views
crime_df.createOrReplaceTempView("crime_df")
crime_closed.createOrReplaceTempView("crime_closed")
crime_stats.createOrReplaceTempView("crime_stats")
total_cases.createOrReplaceTempView("total_cases")
crime_percentage.createOrReplaceTempView("crime_percentage")
crime_top3.createOrReplaceTempView("crime_top3")


sql_query = """
    WITH ranked_data AS (
        SELECT
            cp.year,
            cp.precinct,
            cp.closed_case_rate,
            RANK() OVER (PARTITION BY cp.year ORDER BY cp.closed_case_rate DESC) AS rank
        FROM
            crime_percentage cp
    )
    SELECT
        year,
        precinct,
        closed_case_rate,
        rank
    FROM
        ranked_data
    WHERE
        rank <= 3
    ORDER BY
        year,
        rank
"""


# Execute the SQL query

start_time_sql = time.time()
crime_top3_sql = sc.sql(sql_query)
crime_top3_sql.show()
end_time_sql = time.time()

sql_api_time = end_time_sql - start_time_sql
print(f"Time taken with SQL API : {sql_api_time} seconds")




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|year|   precinct|  closed_case_rate|rank|
+----+-----------+------------------+----+
|2010|    Rampart| 32.84713448949121|   1|
|2010|    Olympic|31.515289821999087|   2|
|2010|     Harbor| 29.36028339237341|   3|
|2011|    Olympic|35.040060090135206|   1|
|2011|    Rampart|  32.4964471814306|   2|
|2011|     Harbor| 28.51336246316431|   3|
|2012|    Olympic| 34.29708533302119|   1|
|2012|    Rampart| 32.46000463714352|   2|
|2012|     Harbor|29.509585848956675|   3|
|2013|    Olympic| 33.58217940999398|   1|
|2013|    Rampart|  32.1060382916053|   2|
|2013|     Harbor|29.723638951488557|   3|
|2014|   Van Nuys|  32.0215235281705|   1|
|2014|West Valley| 31.49754809505847|   2|
|2014|    Mission|31.224939855653567|   3|
|2015|   Van Nuys|32.265140677157845|   1|
|2015|    Mission|30.463762673676303|   2|
|2015|   Foothill|30.353001803658852|   3|
|2016|   Van Nuys|32.194518462124094|   1|
|2016|West Valley| 31.40146437042384|   2|
+----+-----

In [ ]:
# QUERY 2 USING DATAFRAMES:  CSV VS PARQUET FORMAT

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, dense_rank
from pyspark.sql.window import Window
import time

# Δημιουργία Spark Session
sc = SparkSession \
    .builder \
    .appName("Query 2 - CSV vs Parquet") \
    .getOrCreate()

crime_df_2010_2019 = sc.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
crime_df_2020_present = sc.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)

# Ενοποίηση των δεδομένων
crime_df = crime_df_2010_2019.union(crime_df_2020_present)

# Αποθήκευση σε μοναδικό Parquet αρχείο
crime_df.write.mode("overwrite").parquet("s3://groups-bucket-dblab-905418150721/group26/CrimeData.parquet")

print("Dataset saved in Parquet format to S3.")

# Χρονόμετρο για CSV
start_time_csv = time.time()

# Διαβάζουμε τα δεδομένα από CSV
crime_df = sc.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True) \
    .union(sc.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True))

crime_df = crime_df.select(
    col("DATE OCC").alias("date_occ"),
    col("AREA NAME").alias("precinct"),
    col("Status Desc").alias("status")
)

crime_df = crime_df.withColumn("year", col("date_occ").substr(7, 4))
crime_closed = crime_df.filter(col("status").isin("Adult Arrest", "Adult Other", "Juv Arrest", "Juv Other"))

crime_stats = crime_closed.groupBy("year", "precinct") \
    .agg(count("*").alias("closed_cases"))
total_cases = crime_df.groupBy("year", "precinct") \
    .agg(count("*").alias("total_cases"))

crime_percentage = crime_stats.join(total_cases, ["year", "precinct"]) \
    .withColumn("closed_case_rate", (col("closed_cases") / col("total_cases")) * 100)

window_spec = Window.partitionBy("year").orderBy(desc("closed_case_rate"))
crime_top3 = crime_percentage.withColumn("rank", dense_rank().over(window_spec)) \
    .filter(col("rank") <= 3)

crime_top3_sorted = crime_top3.orderBy("year", "rank")
#crime_top3_sorted.show()

end_time_csv = time.time()
csv_time = end_time_csv - start_time_csv
print(f"Time taken with CSV input: {csv_time} seconds")


# Χρονόμετρο για Parquet
start_time_parquet = time.time()

# Διαβάζουμε τα δεδομένα από Parquet
crime_df = sc.read.parquet("s3://groups-bucket-dblab-905418150721/group26/CrimeData.parquet")

crime_df = crime_df.select(
    col("DATE OCC").alias("date_occ"),
    col("AREA NAME").alias("precinct"),
    col("Status Desc").alias("status")
)

crime_df = crime_df.withColumn("year", col("date_occ").substr(7, 4))
crime_closed = crime_df.filter(col("status").isin("Adult Arrest", "Adult Other", "Juv Arrest", "Juv Other"))

crime_stats = crime_closed.groupBy("year", "precinct") \
    .agg(count("*").alias("closed_cases"))
total_cases = crime_df.groupBy("year", "precinct") \
    .agg(count("*").alias("total_cases"))

crime_percentage = crime_stats.join(total_cases, ["year", "precinct"]) \
    .withColumn("closed_case_rate", (col("closed_cases") / col("total_cases")) * 100)

window_spec = Window.partitionBy("year").orderBy(desc("closed_case_rate"))
crime_top3 = crime_percentage.withColumn("rank", dense_rank().over(window_spec)) \
    .filter(col("rank") <= 3)

crime_top3_sorted = crime_top3.orderBy("year", "rank")
#crime_top3_sorted.show()

end_time_parquet = time.time()
parquet_time = end_time_parquet - start_time_parquet
print(f"Time taken with Parquet input: {parquet_time} seconds")


print(f"CSV execution time: {csv_time} seconds")
print(f"Parquet execution time: {parquet_time} seconds")
print(f"Parquet was faster by: {csv_time - parquet_time} seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dataset saved in Parquet format to S3.
Time taken with CSV input: 2.779564619064331 seconds
Time taken with Parquet input: 0.4217860698699951 seconds
CSV execution time: 2.779564619064331 seconds
Parquet execution time: 0.4217860698699951 seconds
Parquet was faster by: 2.357778549194336 seconds

In [ ]:
#QUERY 3 USING DATAFRAMES

In [85]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, regexp_replace, trim, coalesce, lit,explode, when
from pyspark.sql import functions as F

# Initialize SparkSession
sc = SparkSession \
    .builder \
    .appName("Query 3") \
    .getOrCreate()
############################################# AVERAGE INCOME PER PERSON PER AREA ######################################################


######################################################   DATASET PREPROCESSING ########################################################

##  INCOME DATASET ##

income_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv", header=True, inferSchema=True)


income_df_cleaned = income_df.select(
    col("Zip Code").alias("income_zip"),
    regexp_replace(col("Estimated Median Income"), "[$,]", "").cast("int").alias("income"),
    col("Community").alias("area_income")  # Renaming area column to avoid ambiguity
)

#split the records that have multiple areas in the community 
income_split_df = income_df_cleaned.withColumn("area_income", F.explode(split(col("area_income"), r',\s*|\(\s*|\s*\)')))
income_split_df = income_split_df.filter(
    (col("area_income").isNotNull()) & (col("area_income") != "")
)
# Trim whitespace from area names
income_split_df = income_split_df.withColumn("area_income", trim(col("area_income")))
income_split_df= income_split_df.orderBy("income_zip")

##  POPULATION DATASET  ##
population_df = spark.read.option("multiline", "true").json("s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson")

# Explode the 'features' array to access individual feature objects
exploded_df = population_df.select(explode(col("features")).alias("feature"))

cleaned_population_df = exploded_df.select(
    col("feature.properties.COMM").alias("area"),
    col("feature.properties.POP_2010").alias("population"),
    col("feature.properties.ZCTA10").alias("population_zip"),
    col("feature.properties.HOUSING10").alias("households"),
    col("feature.properties.CITY").alias("city")
)

# Filter out rows with null or empty zip codes
cleaned_population_df = cleaned_population_df.filter(
    (col("city") == "Los Angeles") &
    (col("population_zip").isNotNull()) & (col("population_zip") != "") &
    (col("households").isNotNull()) & (col("households") > 0) &
     (col("population").isNotNull()) & (col("population") > 0)
   
)
cleaned_population_df = cleaned_population_df.withColumn("area", trim(col("area")))
cleaned_population_df = cleaned_population_df.filter(
    (col("area").isNotNull()) & (col("area") != "")
)
#cleaned_population_df.show() #This dataframe contains the areas, population, zip and number of households per block


####################################################### AVERAGE HOUSEHOLD SIZE PER AREA ##############################################

#Now i will calculate the total population and total households for the areas with the same name and same zip code. So the total for every zip code./
aggregated_population_df = cleaned_population_df.groupBy("area", "population_zip").agg(
    F.sum("population").alias("total_population"),
    F.sum("households").alias("total_households")
)

#Now i will coacluate the average household size per zip
aggregated_population_df = aggregated_population_df.withColumn(
    "avg_household_size_per_zip",
    when(col("total_households") > 0, col("total_population") / col("total_households")).otherwise(None)
)
sorted_aggregated_population_df =  aggregated_population_df.orderBy("population_zip")
#sorted_aggregated_population_df.show()

#Now i will calculate the average household size per area 
average_household_size_per_area = aggregated_population_df.groupBy("area").agg(
    F.sum((col("avg_household_size_per_zip") * col("total_households"))).alias("weighted_sum"),
    F.sum("total_households").alias("total_households_per_area")
)

average_household_size_per_area = average_household_size_per_area.withColumn(
    "avg_household_size_per_area",
    when(col("total_households_per_area") > 0, col("weighted_sum") / col("total_households_per_area")).otherwise(None)
)

################################################### AVERAGE INCOME PER PERSON PER ZIP ################################################

#For the same area, each zip code has its own mean income so I will first calculate the mean income per person for every zip

joined_df = sorted_aggregated_population_df.join(
    income_split_df,
    (aggregated_population_df["population_zip"] == income_split_df["income_zip"]),
    how="inner"
)


joined_df = joined_df.select(
    "area", "population_zip", "total_population", "total_households",
    "avg_household_size_per_zip", "income"
).orderBy("area", "population_zip")

cleaned_joined_df = joined_df.filter(
    (col("area").isNotNull()) & (col("area") != "")
)
cleaned_joined_df = cleaned_joined_df.filter(
    (col("population_zip").isNotNull()) & (col("population_zip") != "")
)
#cleaned_joined_df.show()


#Now i will calculate the average income per person per area 
joined_df = cleaned_joined_df.withColumn(
    "total_income_per_zip",
    col("income") * col("total_households")
)

aggregated_income_population_df = joined_df.groupBy("area").agg(
    F.sum("total_income_per_zip").alias("total_income_area"),
    F.sum("total_population").alias("total_population_area")
)

aggregated_income_population_df = aggregated_income_population_df.withColumn(
    "avg_income_per_person_per_area",
    when(col("total_population_area") > 0, col("total_income_area") / col("total_population_area")).otherwise(None)
)




########################################################## CRIME RATE PER PERSON #####################################################

######################################################   DATASET PREPROCESSING ########################################################
import sedona
from sedona.spark import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg

# Create Spark Session
spark = SparkSession.builder \
    .appName("GeoJSON read") \
    .getOrCreate()

# Create Sedona context
sedona = SedonaContext.create(spark)

# Read the GeoJSON file from S3
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")

# Formatting the GeoJSON data
flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

# Print schema for the GeoJSON data
#flattened_df.printSchema()

# Read the Crime data files
crime_df_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
crime_df_2020_present = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)
crime_df = crime_df_2010_2019.union(crime_df_2020_present)

# Clean the crime data to extract necessary columns (crime_id, LAT, LON)
df1 = crime_df.select(
    col("DR_NO").alias("crime_id"),  # Crime ID (DR_NO)
    col("LAT").alias("latitude"),  # Latitude
    col("LON").alias("longitude")  # Longitude
)

df1 = df1.withColumn("geom", ST_Point("longitude", "latitude"))

joined_df1 = df1 \
    .join(flattened_df, ST_Within(df1.geom, flattened_df.geometry), "inner")


final_df = joined_df1.select(
    col("COMM").alias("area"),  # Area name (community)
    col("POP_2010").alias("pop10"),  # Population of the block
    col("geometry"),  # Geometry of the block
    col("latitude").alias("latitude"),  # Latitude of the crime location
    col("longitude").alias("longitude"),  # Longitude of the crime location
    col("crime_id")  # Crime ID
)
#final_df.show()

# Calculate the number of crimes per area
crime_count_df = final_df.groupBy("area", "pop10").agg(
    count("crime_id").alias("num_crimes")
)
#This dataframe contain the average crime rate per person per block
crime_rate_df = crime_count_df.withColumn(
    "crime_rate_per_person",
    col("num_crimes") / col("pop10")
)
filtered_df = crime_rate_df.filter(col("area").isNotNull() & (col("area") != ""))

#I will calculate the crime rate per area per person, by finding the average.
average_crime_rate_df = filtered_df.groupBy("area").agg(
    avg("crime_rate_per_person").alias("avg_crime_rate_per_person")
)

############################################# FINAL RESULTS INTO ONE TABLE ###########################################################

final_crime_rate_per_area_per_person = average_crime_rate_df.orderBy("area")


final_result_df = aggregated_income_population_df.join(
    final_crime_rate_per_area_per_person,
    on="area",
    how="inner" 
)

final_result_df = final_result_df.select(
    "area",
    "avg_income_per_person_per_area",
    "avg_crime_rate_per_person"
)

final_result_df.orderBy("area").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------------+-------------------------+
|                area|avg_income_per_person_per_area|avg_crime_rate_per_person|
+--------------------+------------------------------+-------------------------+
|     Adams-Normandie|             8791.458301453711|       1.0866524555163062|
|              Alsace|             11239.50119372442|       1.8363603588772242|
|Angeles National ...|            32657.048780487807|       14.861111111111112|
|    Angelino Heights|            17552.282665401945|       3.5778648514508062|
|              Arleta|            12113.216071836237|       0.6441891411956069|
|     Atwater Village|            28481.236967160792|        2.234442935667145|
|       Baldwin Hills|             17138.06555354499|        3.582901018138237|
|             Bel Air|             63047.61900133641|        0.772383917544476|
|       Beverly Crest|             58638.24134154688|       0.7292138407412925|
|         Beverlywood|              2926

In [41]:
joined_df.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (22)
+- Project (21)
   +- Sort (20)
      +- Exchange (19)
         +- Project (18)
            +- BroadcastHashJoin Inner BuildRight (17)
               :- Project (9)
               :  +- HashAggregate (8)
               :     +- Exchange (7)
               :        +- HashAggregate (6)
               :           +- Project (5)
               :              +- Filter (4)
               :                 +- Generate (3)
               :                    +- Filter (2)
               :                       +- Scan json  (1)
               +- BroadcastExchange (16)
                  +- Project (15)
                     +- Filter (14)
                        +- Generate (13)
                           +- Project (12)
                              +- Filter (11)
                                 +- Scan csv  (10)


(1) Scan json 
Output [1]: [features#2157]
Batched: false
Location: InMemoryFileIndex [s3://initial-notebook-data-bucket-dblab-905418150

In [42]:
joined_df1.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
RangeJoin (13)
:- Union (7)
:  :- Project (3)
:  :  +- Filter (2)
:  :     +- Scan csv  (1)
:  +- Project (6)
:     +- Filter (5)
:        +- Scan csv  (4)
+- * Project (12)
   +- * Filter (11)
      +- * Generate (10)
         +- * Filter (9)
            +- Scan geojson  (8)


(1) Scan csv 
Output [3]: [DR_NO#2451, LAT#2477, LON#2478]
Batched: false
Location: InMemoryFileIndex [s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv]
ReadSchema: struct<DR_NO:int,LAT:double,LON:double>

(2) Filter
Input [3]: [DR_NO#2451, LAT#2477, LON#2478]
Condition : isnotnull( **org.apache.spark.sql.sedona_sql.expressions.ST_Point**  )

(3) Project
Output [4]: [DR_NO#2451 AS crime_id#2609, LAT#2477 AS latitude#2610, LON#2478 AS longitude#2611,  **org.apache.spark.sql.sedona_sql.expressions.ST_Point**   AS geom#2615]
Input [3]: [DR_NO#2451, LAT#2477, LON#2478]

(4) Scan csv 
Output [3]: [DR_NO#2525, LAT#2551, LON#2552]
Batched: fals

In [43]:
final_result_df.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (46)
+- Project (45)
   +- SortMergeJoin Inner (44)
      :- Sort (22)
      :  +- Project (21)
      :     +- HashAggregate (20)
      :        +- Exchange (19)
      :           +- HashAggregate (18)
      :              +- Project (17)
      :                 +- BroadcastHashJoin Inner BuildRight (16)
      :                    :- HashAggregate (8)
      :                    :  +- Exchange (7)
      :                    :     +- HashAggregate (6)
      :                    :        +- Project (5)
      :                    :           +- Filter (4)
      :                    :              +- Generate (3)
      :                    :                 +- Filter (2)
      :                    :                    +- Scan json  (1)
      :                    +- BroadcastExchange (15)
      :                       +- Project (14)
      :                          +- Filter (13)
      :                             +- Generate (12)
      :              

In [74]:
import time

join_methods = ["broadcast", "merge", "shuffle_hash", "shuffle_replicate_nl"]
results = []

for method in join_methods:
    start_time = time.time()
    
    joined_df = sorted_aggregated_population_df.hint(method).join(
        income_split_df.hint(method),
        (sorted_aggregated_population_df["population_zip"] == income_split_df["income_zip"]),
        how="inner"
    )
    
    # Force execution and measure time
    joined_df.explain()  # Show the execution plan
    count = joined_df.count()
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    results.append(("Income-Population Join", method, execution_time, count))
    print(f"Method: {method} | Execution Time: {execution_time:.4f} seconds | Row Count: {count}\n")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [cast(population_zip#772 as int)], [income_zip#742], Inner, BuildRight, false
   :- Project [area#785, population_zip#772, total_population#797L, total_households#799L, CASE WHEN (total_households#799L > 0) THEN (cast(total_population#797L as double) / cast(total_households#799L as double)) END AS avg_household_size_per_zip#804]
   :  +- HashAggregate(keys=[area#785, population_zip#772], functions=[sum(population#771L), sum(households#773L)], schema specialized)
   :     +- Exchange hashpartitioning(area#785, population_zip#772, 1000), ENSURE_REQUIREMENTS, [plan_id=9773]
   :        +- HashAggregate(keys=[area#785, population_zip#772], functions=[partial_sum(population#771L), partial_sum(households#773L)], schema specialized)
   :           +- Project [trim(feature#767.properties.COMM, None) AS area#785, feature#767.properties.POP_2010 AS population#771L, feature#767.properties.ZCTA10 AS population_zip#772, fe

In [ ]:
#QUERY 4

In [75]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
369,application_1738075734771_0370,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
270,application_1738075734771_0271,pyspark,idle,Link,Link,None,
284,application_1738075734771_0285,pyspark,idle,Link,Link,None,
285,application_1738075734771_0286,pyspark,idle,Link,Link,None,
300,application_1738075734771_0301,pyspark,idle,Link,Link,None,
326,application_1738075734771_0327,pyspark,busy,Link,Link,None,
330,application_1738075734771_0331,pyspark,idle,Link,Link,None,
332,application_1738075734771_0333,pyspark,idle,Link,Link,None,
334,application_1738075734771_0335,pyspark,idle,Link,Link,None,
343,application_1738075734771_0344,pyspark,idle,Link,Link,None,
353,application_1738075734771_0354,pyspark,idle,Link,Link,None,


In [79]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
371,application_1738075734771_0372,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
270,application_1738075734771_0271,pyspark,idle,Link,Link,None,
284,application_1738075734771_0285,pyspark,idle,Link,Link,None,
285,application_1738075734771_0286,pyspark,idle,Link,Link,None,
300,application_1738075734771_0301,pyspark,idle,Link,Link,None,
326,application_1738075734771_0327,pyspark,idle,Link,Link,None,
330,application_1738075734771_0331,pyspark,idle,Link,Link,None,
332,application_1738075734771_0333,pyspark,idle,Link,Link,None,
334,application_1738075734771_0335,pyspark,idle,Link,Link,None,
343,application_1738075734771_0344,pyspark,idle,Link,Link,None,
353,application_1738075734771_0354,pyspark,idle,Link,Link,None,


In [84]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
373,application_1738075734771_0374,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
270,application_1738075734771_0271,pyspark,idle,Link,Link,None,
284,application_1738075734771_0285,pyspark,idle,Link,Link,None,
285,application_1738075734771_0286,pyspark,idle,Link,Link,None,
300,application_1738075734771_0301,pyspark,idle,Link,Link,None,
326,application_1738075734771_0327,pyspark,idle,Link,Link,None,
330,application_1738075734771_0331,pyspark,idle,Link,Link,None,
332,application_1738075734771_0333,pyspark,idle,Link,Link,None,
334,application_1738075734771_0335,pyspark,idle,Link,Link,None,
343,application_1738075734771_0344,pyspark,idle,Link,Link,None,
353,application_1738075734771_0354,pyspark,idle,Link,Link,None,


In [86]:
# Query 4 with DataFrame
import time
import sedona
from sedona.spark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col, to_timestamp, year, count

sedona = SedonaContext.create(spark)

spark = SparkSession \
    .builder \
    .appName("DF query 4 execution") \
    .getOrCreate()

start_time = time.time()
crimedata = StructType([
    StructField("DR_NO", StringType(), True),
    StructField("Date Rptd", StringType(), True),
    StructField("DATE OCC", StringType(), True),
    StructField("TIME OCC", StringType(), True),
    StructField("AREA", IntegerType(), True),
    StructField("AREA NAME", StringType(), True),
    StructField("Rpt Dist No", IntegerType(), True),
    StructField("Part 1-2", IntegerType(), True),
    StructField("Crm Cd", IntegerType(), True),
    StructField("Crm Cd Desc", StringType(), True),
    StructField("Mocodes", StringType(), True),
    StructField("Vict Age", IntegerType(), True),
    StructField("Vict Sex", StringType(), True),
    StructField("Vict Descent", StringType(), True),
    StructField("Premis Cd", IntegerType(), True),
    StructField("Premis Desc", StringType(), True),
    StructField("Weapon Used Cd", IntegerType(), True),
    StructField("Weapon Desc", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Status Desc", StringType(), True),
    StructField("Crm Cd 1", IntegerType(), True),
    StructField("Crm Cd 2", IntegerType(), True),
    StructField("Crm Cd 3", IntegerType(), True),
    StructField("Crm Cd 4", IntegerType(), True),
    StructField("LOCATION", StringType(), True),
    StructField("Cross Street", StringType(), True),
    StructField("LAT", FloatType(), True),
    StructField("LON", FloatType(), True)
])

REdata = StructType([
    StructField("Vict_Desc", StringType(), True),
    StructField("Vict_Desc_Full", StringType(), True),
])


#we checked crime data from 2020 and on, and there were no incidents reported from 2015. Thus, we only used the one set
crimedata_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=False, schema=crimedata)
RE_codes = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv", header=False, schema=REdata)

crime_df_2015 = crimedata_2010_2019 \
    .withColumn("DATE OCC", to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")) \
    .withColumn("DATE OCC", year(col("DATE OCC"))) \
    .filter(col("DATE OCC") == 2015) \
    .filter(col("Vict Descent").isNotNull())

crime_df_2015 = crime_df_2015.select(
    col("DATE OCC").alias("date_occ"),
    col("AREA NAME").alias("precinct"),
    col("Vict Descent").alias("vict_descent"),
    col("DR_NO").alias("crime_id"),
    col("LAT").alias("latitude"),
    col("LON").alias("longitude")
).withColumn("geom", ST_Point("longitude", "latitude"))

# Top 3 areas by income excluding NULL and 0.0 values
top_3_areas_by_income = aggregated_income_population_df.filter(
    col("avg_income_per_person_per_area").isNotNull() & (col("avg_income_per_person_per_area") > 0.0)).orderBy(
    col("avg_income_per_person_per_area").desc()).limit(3)
#top_3_areas_by_income.select("area", "avg_income_per_person_per_area").show()

# Bottom 3 areas by income excluding NULL and 0.0 values
bottom_3_areas_by_income = aggregated_income_population_df.filter(
    col("avg_income_per_person_per_area").isNotNull() & (col("avg_income_per_person_per_area") > 0.0)).orderBy(
    col("avg_income_per_person_per_area").asc()).limit(3)
#bottom_3_areas_by_income.select("area", "avg_income_per_person_per_area").show()

joined_df_4 = crime_df_2015.join(flattened_df, ST_Within(crime_df_2015.geom, flattened_df.geometry), "inner")

# Get a list of top and bottom areas
top_areas = [row['area'] for row in top_3_areas_by_income.collect()]
bottom_areas = [row['area'] for row in bottom_3_areas_by_income.collect()]

final_df_4 = joined_df_4.select(
    col("COMM").alias("area"),
    col("precinct").alias("precinct"),
    col("date_occ").alias("date_occ"),
    col("geometry"),
    col("latitude").alias("latitude"),
    col("longitude").alias("longitude"),
    col("vict_descent")  # Crime ID
)

crime_top_areas = final_df_4.filter(col("area").isin(top_areas))
# crime_top_areas.show()
crime_bottom_areas = final_df_4.filter(col("area").isin(bottom_areas))
# crime_bottom_areas.show()

# Join with RE_codes to get the full description
crime_top_areas_with_re = crime_top_areas.join(RE_codes, crime_top_areas["vict_descent"] == RE_codes["Vict_Desc"], "left")
crime_bottom_areas_with_re = crime_bottom_areas.join(RE_codes, crime_bottom_areas["vict_descent"] == RE_codes["Vict_Desc"], "left")
#crime_top_areas_with_re.show(5)

# Group by Victim Descent and count the number of victims
top_areas_victim_count = crime_top_areas_with_re.groupBy("Vict_Desc_Full").agg(count("*").alias("victim_count")).orderBy(col("victim_count").desc())
bottom_areas_victim_count = crime_bottom_areas_with_re.groupBy("Vict_Desc_Full").agg(count("*").alias("victim_count")).orderBy(col("victim_count").desc())

# Show the results
print("Top 3 Areas by Income - Victim Descent Count:")
top_areas_victim_count.show()

print("Bottom 3 Areas by Income - Victim Descent Count:")
bottom_areas_victim_count.show()

end_time = time.time()
ex_time = end_time - start_time
print(f"Time taken: {ex_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Top 3 Areas by Income - Victim Descent Count:
+--------------------+------------+
|      Vict_Desc_Full|victim_count|
+--------------------+------------+
|               White|         695|
|               Other|          86|
|Hispanic/Latin/Me...|          77|
|             Unknown|          49|
|               Black|          43|
|         Other Asian|          22|
|             Chinese|           1|
|American Indian/A...|           1|
+--------------------+------------+

Bottom 3 Areas by Income - Victim Descent Count:
+--------------------+------------+
|      Vict_Desc_Full|victim_count|
+--------------------+------------+
|Hispanic/Latin/Me...|        3215|
|               Black|         872|
|               White|         431|
|               Other|         265|
|         Other Asian|         142|
|             Unknown|          26|
|American Indian/A...|          24|
|              Korean|           5|
|             Chinese|           3|
|            Filipino|           2|
|   

In [78]:
print(f"Time taken when using 2 executors 1 core/2GB memory: {ex_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken when using 2 executors 1 core/2GB memory: 83.48 seconds

In [83]:
print(f"Time taken when using 2 executors 2 core/4GB memory: {ex_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken when using 2 executors 2 core/4GB memory: 64.38 seconds

In [87]:
print(f"Time taken when using 2 executors 4 core/8GB memory: {ex_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken when using 2 executors 4 core/8GB memory: 59.17 seconds

In [52]:
final_df_4.select("area").distinct() \
    .filter(col("area").startswith("M")) \
    .orderBy("area") \
    .show()

# Show distinct areas from cleaned_population_df that start with 'M', sorted by area
cleaned_population_df.select("area").distinct() \
    .filter(col("area").startswith("M")) \
    .orderBy("area") \
    .show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|             area|
+-----------------+
|Manchester Square|
|Mandeville Canyon|
|        Mar Vista|
| Marina Peninsula|
|   Marina del Rey|
|          Melrose|
|         Mid-city|
|     Miracle Mile|
|    Mission Hills|
|   Mt. Washington|
+-----------------+

+-----------------+
|             area|
+-----------------+
|Manchester Square|
|Mandeville Canyon|
|        Mar Vista|
| Marina Peninsula|
|          Melrose|
|         Mid-city|
|     Miracle Mile|
|    Mission Hills|
|   Mt. Washington|
+-----------------+

In [62]:
##############################  QUERY 5 #################################
import time
from pyspark.sql.functions import format_number
from pyspark.sql.window import Window

start_time = time.time()
police_stations = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv", header=True, inferSchema=True)
police_stations = police_stations.withColumn('X', format_number('X', 10))  # Adjust the number of decimals as needed
police_stations = police_stations.withColumn('Y', format_number('Y', 10))
police_stations_df = police_stations.withColumn("geom", ST_Point(col("X"), col("Y")))
crime_df = df1.filter(
    (F.col("longitude") != 0) & (F.col("latitude") != 0)
)
crime_df = crime_df.withColumnRenamed("geom", "crime_geom")
police_stations_df = police_stations_df.withColumnRenamed("geom", "police_geom")

joined_df = crime_df.crossJoin(police_stations_df)
joined_df = joined_df.withColumn("distance", ST_DistanceSphere(col("crime_geom"), col("police_geom"))/1000)
joined_df = joined_df.select(
     col("crime_id").alias("crime_id"),
    col("distance").alias("distance"),
    col("FID").alias("police id"),
    col("DIVISION").alias("division")
)

joined_df = joined_df.orderBy("crime_id")
window_spec = Window.partitionBy("crime_id").orderBy("distance")
df_with_rank = joined_df.withColumn("rank", F.row_number().over(window_spec))
filtered_df = df_with_rank.filter(F.col("rank") == 1)
filtered_df = filtered_df.orderBy("police id")
crime_count_and_avg_distance = filtered_df.groupBy("police id", "division").agg(
    F.count("crime_id").alias("crime_count"),
    F.avg("distance").alias("avg_distance")
)
crime_count_and_avg_distance_sorted = crime_count_and_avg_distance.orderBy(F.col("crime_count").desc())

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")

crime_count_and_avg_distance_sorted.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution Time: 0.7246222496032715 seconds
+---------+----------------+-----------+------------------+
|police id|        division|crime_count|      avg_distance|
+---------+----------------+-----------+------------------+
|       13|       HOLLYWOOD|     224340|2.0762639601788018|
|       16|        VAN NUYS|     210134|2.9533697428211574|
|        5|       SOUTHWEST|     188901| 2.191398805778549|
|       10|        WILSHIRE|     185996|2.5926655329791277|
|        3|     77TH STREET|     171827|1.7165449719708101|
|       11|         OLYMPIC|     170897|1.7236036971777815|
|       15| NORTH HOLLYWOOD|     167854| 2.643006094141875|
|        4|         PACIFIC|     161359| 3.850070655306162|
|        7|         CENTRAL|     153871| 0.992476437456902|
|       12|         RAMPART|     152736|1.5345341879190026|
|        2|       SOUTHEAST|     152176| 2.421866215887735|
|       17|     WEST VALLEY|     138643| 3.035671216314056|
|       18|         TOPANGA|     138217|3.296954841755783

In [54]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
30,application_1738075734771_0031,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1738075734771_0012,pyspark,idle,Link,Link,None,
14,application_1738075734771_0015,pyspark,idle,Link,Link,None,
15,application_1738075734771_0016,pyspark,idle,Link,Link,None,
18,application_1738075734771_0019,pyspark,idle,Link,Link,None,
21,application_1738075734771_0022,pyspark,idle,Link,Link,None,
22,application_1738075734771_0023,pyspark,idle,Link,Link,None,
26,application_1738075734771_0027,pyspark,idle,Link,Link,None,
30,application_1738075734771_0031,pyspark,idle,Link,Link,None,✔


In [ ]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2",
        "spark.driver.memory": "2g"
    }
}

In [ ]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "8",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1",
        "spark.driver.memory": "2g"
    }
}